<a href="https://colab.research.google.com/github/hariharankpz/Playground/blob/master/Assignments/Top%20Selling%20product%20for%20a%20group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Test_Scenario1.xls to Test_Scenario1.xls


In [ ]:
# !pip install xlrd==1.2.0
# !pip install pandasql
# !pip install --upgrade sqlite
# !pip install pysqlite3

Importing neccessary packages

In [ ]:
import pandas as pd
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


In [ ]:
df = pd.read_excel("Test_Scenario1.xls")
df.head()


,CUSTOMER_NUMBER,AGE,COUNTRY,MARITAL_STATUS,PRODUCT_CODE,PURCHASE_DATE,TRANS_NUM,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,1002,113.0,IN,NaN,VOYCC,2019-08-09,ACT633528,NaN,NaN,NaN,NaN,NaN
1,10096,30.0,IN,M,SD,2019-04-01,ACT623150,NaN,NaN,NaN,NaN,NaN
2,10096,30.0,IN,M,SD,2019-07-01,ACT628206,NaN,NaN,NaN,NaN,NaN
3,10096,30.0,IN,M,SD,2019-07-02,ACT628207,NaN,NaN,NaN,NaN,NaN
4,10099,19.0,AF,NaN,SD,2019-02-07,ACT609618,NaN,NaN,NaN,Data Set Description,NaN


In [ ]:
df1 = df.copy()

Handling missing values by replacing nan values with mean of ages

In [ ]:
df1["AGE"].fillna(df["AGE"].mean(),inplace=True)

In [ ]:
df1.AGE.unique()

array([113.        ,  30.        ,  19.        ,  57.50235627,
        22.        ,  52.        ,   4.        ,  25.        ,
       115.        , 111.        ,  64.        ,  70.        ,
       117.        , 114.        ,  29.        ,  33.        ,
        51.        ,  75.        , 110.        ,  78.        ,
        38.        ,  34.        ,  99.        ,  36.        ,
        59.        ,  40.        ,  80.        ,  27.        ,
        20.        , 112.        , 120.        ,   3.        ,
       116.        ,  26.        ,  35.        ,  17.        ,
        60.        ,   5.        ,  61.        ,  45.        ,
       118.        ,  68.        ,  10.        ,  32.        ,
        82.        ,  63.        ,  43.        ,  24.        ,
        58.        , 124.        ,  21.        ,   1.        ,
       119.        , 130.        , 122.        , 126.        ])

Checking whether there is any null values present in AGE column

In [ ]:
df1.AGE.isnull().values.any()

False

Since there is PURCHASED_QUANTITY column available as per the question, generating column values with random numbers between 5 and 15 


In [ ]:
df1['PURCHASED_QUANTITY'] = np.random.randint(5,15, size=len(df1))

Grouping the ages into 5 groups

In [ ]:
df1.loc[df1['AGE']<=12, 'AGE_GROUP'] = '0-12'
df1.loc[df1['AGE'].between(13,19), 'AGE_GROUP'] = '13-19'
df1.loc[df1['AGE'].between(20,39), 'AGE_GROUP'] = '20-39'
df1.loc[df1['AGE'].between(40,65), 'AGE_GROUP'] = '40-65'
df1.loc[df1['AGE']>65, 'AGE_GROUP'] = '65+'

In [ ]:
df2 = df1[["CUSTOMER_NUMBER", "AGE", "COUNTRY", "MARITAL_STATUS", "PRODUCT_CODE",
       "PURCHASE_DATE", "TRANS_NUM","AGE_GROUP", "PURCHASED_QUANTITY"]]

In [ ]:
df2.groupby('AGE_GROUP').apply(lambda x: x.sort_values(["PURCHASED_QUANTITY"], ascending = False)).reset_index(drop=True)

,CUSTOMER_NUMBER,AGE,COUNTRY,MARITAL_STATUS,PRODUCT_CODE,PURCHASE_DATE,TRANS_NUM,AGE_GROUP,PURCHASED_QUANTITY
0,59373,1.0,IN,S,SD,2019-08-27,ACT636503,0-12,14
1,57907,3.0,IN,NaN,SD,2019-08-02,ACT635780,0-12,14
2,57908,3.0,IN,NaN,SD,2019-08-02,ACT632143,0-12,14
3,38704,10.0,IN,NaN,SD,2019-01-02,ACT618423,0-12,14
4,38704,10.0,IN,NaN,SD,2019-02-18,ACT623028,0-12,14
...,...,...,...,...,...,...,...,...,...
2570,36297,68.0,IN,NaN,SD,2019-08-27,ACT653618,65+,5
2571,64567,130.0,IN,NaN,RNC,2019-10-13,ACT651054,65+,5
2572,55098,70.0,IN,NaN,SD,2019-06-06,ACT626417,65+,5
2573,38897,82.0,IN,NaN,AGR,2019-09-03,ACT644719,65+,5


In [ ]:
import pandasql as ps


Applying window functions to find out top 3 prodcuts in terms highest purchased quantity

In [ ]:
output_df  = ps.sqldf(
                        """ SELECT AGE_GROUP,PRODUCT_CODE,rank1 FROM
                       (SELECT AGE_GROUP,PRODUCT_CODE, row_number() OVER(PARTITION BY AGE_GROUP ORDER BY PURCHASED_QUANTITY DESC) as rank1 FROM df2) as temp
                       WHERE rank1<=3   
                        """
                      )

Saving output to as csv file

In [ ]:
output_df.to_csv("Top_Selling_product_for_a_group.csv",index=False)
output_df

,AGE_GROUP,PRODUCT_CODE,rank1
0,0-12,SD,1
1,0-12,SD,2
2,0-12,SD,3
3,13-19,SD,1
4,13-19,AV,2
5,13-19,SD,3
6,20-39,SD,1
7,20-39,SD,2
8,20-39,BD,3
9,40-65,SD,1


In [ ]:
import sqlite3
sqlite3.sqlite_version

'3.22.0'

In [ ]:
!add-apt-repository -y ppa:sergey-dryabzhinsky/packages
!apt update
!apt install sqlite3

import sqlite3
print(sqlite3.sqlite_version)

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu